In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('white')

In [4]:
prices = pd.read_csv('data/raw_sec_data.csv')
prices.set_index(['ticker', 'date'], inplace=True)
prices = prices.dropna()
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53874 entries, ('XLK', '1998-12-22') to ('XLI', '2022-10-05')
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       53874 non-null  float64
 1   High       53874 non-null  float64
 2   Low        53874 non-null  float64
 3   Close      53874 non-null  float64
 4   Adj Close  53874 non-null  float64
 5   Volume     53874 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 2.8+ MB


In [5]:
prices = (prices.unstack('ticker')
        .sort_index()
        .ffill(limit=5)
        .dropna(axis=1)
        .stack('ticker')
        .swaplevel())
prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53874 entries, ('XLB', '1998-12-22') to ('XLY', '2022-10-05')
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       53874 non-null  float64
 1   High       53874 non-null  float64
 2   Low        53874 non-null  float64
 3   Close      53874 non-null  float64
 4   Adj Close  53874 non-null  float64
 5   Volume     53874 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 2.8+ MB


## Feature Engineering

### Compute period returns

In [6]:
intervals = [1, 5, 10, 21, 63]

In [7]:
returns = []
by_ticker = prices.groupby(level='ticker').Close
for t in intervals:
    returns.append(by_ticker.pct_change(t).to_frame(f'ret_{t}'))
returns = pd.concat(returns, axis=1)

In [8]:
returns.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53874 entries, ('XLB', '1998-12-22') to ('XLY', '2022-10-05')
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ret_1   53865 non-null  float64
 1   ret_5   53829 non-null  float64
 2   ret_10  53784 non-null  float64
 3   ret_21  53685 non-null  float64
 4   ret_63  53307 non-null  float64
dtypes: float64(5)
memory usage: 2.4+ MB


### Remove outliers

In [9]:
max_ret_by_sym = returns.groupby(level='ticker').max()

In [10]:
percentiles = [0.001, .005, .01, .025, .05, .1]
percentiles += [1-p for p in percentiles]
max_ret_by_sym.describe(percentiles=sorted(percentiles)[6:])

,ret_1,ret_5,ret_10,ret_21,ret_63
count,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.132090,0.233845,0.291355,0.342143,0.518607
std,0.028726,0.061107,0.102232,0.097633,0.220854
min,0.085106,0.143327,0.180547,0.205223,0.251223
50%,0.127934,0.225138,0.241709,0.312041,0.504035
90%,0.164645,0.297737,0.399038,0.446258,0.730907
95%,0.164696,0.322990,0.456707,0.499571,0.862217
97.5%,0.164721,0.335616,0.485542,0.526228,0.927872
99%,0.164736,0.343192,0.502843,0.542222,0.967265
99.5%,0.164741,0.345717,0.508610,0.547553,0.980396


In [12]:
quantiles = max_ret_by_sym.quantile(.95)
to_drop = []
for ret, q in quantiles.items():
    to_drop.extend(max_ret_by_sym[max_ret_by_sym[ret]>q].index.tolist()) 

### Calculate relative return percentiles

In [14]:
returns = []
by_sym = prices.groupby(level='ticker').Close
for t in intervals:
    ret = by_sym.pct_change(t)
    rel_perc = (ret.groupby(level='date')
             .apply(lambda x: pd.qcut(x, q=5, labels=False, duplicates='drop')))
    returns.extend([ret.to_frame(f'ret_{t}'), rel_perc.to_frame(f'ret_rel_perc_{t}')])
returns = pd.concat(returns, axis=1)

In [15]:
returns.tail(15)

,,ret_1,ret_rel_perc_1,ret_5,ret_rel_perc_5,ret_10,ret_rel_perc_10,ret_21,ret_rel_perc_21,ret_63,ret_rel_perc_63
ticker,date,,,,,,,,,,
XLI,2022-10-04,0.034802,2.0,0.056592,3.0,-0.006301,3.0,-0.041775,2.0,0.011801,3.0
XLK,2022-10-04,0.033562,1.0,0.028690,1.0,-0.016397,1.0,-0.047271,1.0,-0.027282,0.0
XLP,2022-10-04,0.015174,0.0,0.009966,1.0,-0.028753,1.0,-0.050303,1.0,-0.055510,0.0
XLU,2022-10-04,0.021207,0.0,0.000145,0.0,-0.067822,0.0,-0.076944,0.0,-0.017969,1.0
XLV,2022-10-04,0.023220,1.0,0.043568,2.0,0.013544,4.0,0.011841,4.0,-0.025655,1.0
XLY,2022-10-04,0.034925,3.0,0.006960,0.0,-0.060841,0.0,-0.050936,0.0,0.034345,4.0
XLB,2022-10-05,-0.011264,0.0,0.040023,4.0,0.017961,3.0,-0.028872,3.0,-0.023868,2.0
XLE,2022-10-05,0.020665,4.0,0.115703,4.0,0.066210,4.0,0.022469,4.0,0.141166,4.0
XLF,2022-10-05,-0.007730,1.0,0.034160,3.0,0.000000,1.0,-0.021945,3.0,-0.001866,3.0


### Technical Indicators

#### Percentage Price Oscillator


In [16]:
ppo = prices.groupby(level='ticker').Close.apply(talib.PPO).to_frame('PPO')

#### Normalized Average True Range


In [17]:
natr = prices.groupby(level='ticker', group_keys=False).apply(lambda x: talib.NATR(x.High, x.Low, x.Close)).to_frame('NATR')

#### Relative Strength Indicator

In [18]:
rsi = prices.groupby(level='ticker').Close.apply(talib.RSI).to_frame('RSI')

#### Bollinger Bands

In [19]:
def get_bollinger(x):
    u, m, l = talib.BBANDS(x)
    return pd.DataFrame({'u': u, 'm': m, 'l': l})

In [20]:
bbands = prices.groupby(level='ticker').Close.apply(get_bollinger)

### Combine Features

In [21]:
data = pd.concat([prices, returns, ppo, natr, rsi, bbands], axis=1)

In [22]:
data['bbl'] = data.Close.div(data.l)
data['bbu'] = data.u.div(data.Close)
data = data.drop(['u', 'm', 'l'], axis=1)

In [23]:
data

Open        High         Low       Close   Adj Close  \
ticker date                                                                     
XLB    1998-12-22   20.781250   20.828125   20.750000   20.828125   12.329891   
       1998-12-23   20.968750   21.140625   20.968750   21.046875   12.459377   
       1998-12-24   21.296875   21.531250   21.203125   21.531250   12.746130   
       1998-12-28   21.500000   21.500000   21.312500   21.343750   12.635130   
       1998-12-29   21.734375   21.734375   21.734375   21.734375   12.866365   
...                       ...         ...         ...         ...         ...   
XLY    2022-09-29  148.130005  148.330002  143.910004  145.309998  145.309998   
       2022-09-30  143.610001  146.539993  142.039993  142.449997  142.449997   
       2022-10-03  142.320007  143.910004  140.199997  142.589996  142.589996   
       2022-10-04  146.130005  149.009995  145.710007  147.570007  147.570007   
       2022-10-05  145.229996  147.669998  143.679993  146.699997  146.699997   

                    Volume     ret_1  ret_rel_perc_1     ret_5  \
ticker date                                                      
XLB    1998-12-22     1900       NaN             NaN       NaN   
       1998-12-23      700  0.010503             1.0       NaN   
       1998-12-24     1500  0.023014             4.0       NaN   
       1998-12-28     2500 -0.008708             1.0       NaN   
       1998-12-29      100  0.018302             3.0       NaN   
...                    ...       ...             ...       ...   
XLY    2022-09-29  6148298 -0.034549             0.0 -0.030426   
       2022-09-30  6116165 -0.019682             0.0 -0.027114   
       2022-10-03  8645800  0.000983             0.0 -0.023958   
       2022-10-04  9831437  0.034925             3.0  0.006960   
       2022-10-05  6269800 -0.005896             1.0 -0.025314   

                   ret_rel_perc_5  ...  ret_rel_perc_10    ret_21  \
ticker date                        ...                              
XLB    1998-12-22             NaN  ...              NaN       NaN   
       1998-12-23             NaN  ...              NaN       NaN   
       1998-12-24             NaN  ...              NaN       NaN   
       1998-12-28             NaN  ...              NaN       NaN   
       1998-12-29             NaN  ...              NaN       NaN   
...                           ...  ...              ...       ...   
XLY    2022-09-29             2.0  ...              1.0 -0.074812   
       2022-09-30             1.0  ...              0.0 -0.083982   
       2022-10-03             0.0  ...              0.0 -0.091378   
       2022-10-04             0.0  ...              0.0 -0.050936   
       2022-10-05             0.0  ...              0.0 -0.054585   

                   ret_rel_perc_21    ret_63  ret_rel_perc_63       PPO  \
ticker date                                                               
XLB    1998-12-22              NaN       NaN              NaN       NaN   
       1998-12-23              NaN       NaN              NaN       NaN   
       1998-12-24              NaN       NaN              NaN       NaN   
       1998-12-28              NaN       NaN              NaN       NaN   
       1998-12-29              NaN       NaN              NaN       NaN   
...                            ...       ...              ...       ...   
XLY    2022-09-29              4.0  0.056954              4.0 -2.491272   
       2022-09-30              3.0  0.017427              4.0 -2.912506   
       2022-10-03              0.0 -0.003842              4.0 -3.242907   
       2022-10-04              0.0  0.034345              4.0 -3.510721   
       2022-10-05              0.0  0.002391              4.0 -3.929632   

                       NATR        RSI       bbl       bbu  
ticker date                                                 
XLB    1998-12-22       NaN        NaN       NaN       NaN  
       1998-12-23       NaN        NaN       NaN       

In [24]:
data = data.drop(prices.columns, axis=1)
data = data.reset_index(level='date')
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date', append=True)
data

ret_1  ret_rel_perc_1     ret_5  ret_rel_perc_5  \
ticker date                                                             
XLB    1998-12-22       NaN             NaN       NaN             NaN   
       1998-12-23  0.010503             1.0       NaN             NaN   
       1998-12-24  0.023014             4.0       NaN             NaN   
       1998-12-28 -0.008708             1.0       NaN             NaN   
       1998-12-29  0.018302             3.0       NaN             NaN   
...                     ...             ...       ...             ...   
XLY    2022-09-29 -0.034549             0.0 -0.030426             2.0   
       2022-09-30 -0.019682             0.0 -0.027114             1.0   
       2022-10-03  0.000983             0.0 -0.023958             0.0   
       2022-10-04  0.034925             3.0  0.006960             0.0   
       2022-10-05 -0.005896             1.0 -0.025314             0.0   

                     ret_10  ret_rel_perc_10    ret_21  ret_rel_perc_21  \
ticker date                                                               
XLB    1998-12-22       NaN              NaN       NaN              NaN   
       1998-12-23       NaN              NaN       NaN              NaN   
       1998-12-24       NaN              NaN       NaN              NaN   
       1998-12-28       NaN              NaN       NaN              NaN   
       1998-12-29       NaN              NaN       NaN              NaN   
...                     ...              ...       ...              ...   
XLY    2022-09-29 -0.087363              1.0 -0.074812              4.0   
       2022-09-30 -0.098075              0.0 -0.083982              3.0   
       2022-10-03 -0.107920              0.0 -0.091378              0.0   
       2022-10-04 -0.060841              0.0 -0.050936              0.0   
       2022-10-05 -0.043739              0.0 -0.054585              0.0   

                     ret_63  ret_rel_perc_63       PPO      NATR        RSI  \
ticker date                                                                   
XLB    1998-12-22       NaN              NaN       NaN       NaN        NaN   
       1998-12-23       NaN              NaN       NaN       NaN        NaN   
       1998-12-24       NaN              NaN       NaN       NaN        NaN   
       1998-12-28       NaN              NaN       NaN       NaN        NaN   
       1998-12-29       NaN              NaN       NaN       NaN        NaN   
...                     ...              ...       ...       ...        ...   
XLY    2022-09-29  0.056954              4.0 -2.491272  3.054895  35.540481   
       2022-09-30  0.017427              4.0 -2.912506  3.119284  32.810902   
       2022-10-03 -0.003842              4.0 -3.242907  3.079482  33.081835   
       2022-10-04  0.034345              4.0 -3.510721  3.073768  42.035720   
       2022-10-05  0.002391              4.0 -3.929632  3.065414  41.003512   

                        bbl       bbu  
ticker date                            
XLB    1998-12-22       NaN       NaN  
       1998-12-23       NaN       NaN  
       1998-12-24       NaN       NaN  
       1998-12-28       NaN       NaN  
       1998-12-29  1.052754  1.009852  
...                     ...       ...  
XLY    2022-09-29  1.013755  1.036499  
       2022-09-30  1.010282  1.062575  
       2022-10-03  1.021795  1.061894  
       2022-10-04  1.057449  1.028794  
       2022-10-05  1.042341  1.016408  

[53874 rows x 15 columns]

In [26]:
dates = data.index.get_level_values('date')
data['weekday'] = dates.weekday
data['month'] = dates.month
data['year'] = dates.year

## Compute forward returns

In [27]:
outcomes = []
by_ticker = data.groupby('ticker')
for t in intervals:
    k = f'fwd_ret_{t:02}'
    outcomes.append(k)
    data[k] = by_ticker[f'ret_{1}'].shift(-t)

In [32]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53874 entries, ('XLB', Timestamp('1998-12-22 00:00:00')) to ('XLY', Timestamp('2022-10-05 00:00:00'))
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ret_1            53865 non-null  float64
 1   ret_rel_perc_1   53865 non-null  float64
 2   ret_5            53829 non-null  float64
 3   ret_rel_perc_5   53829 non-null  float64
 4   ret_10           53784 non-null  float64
 5   ret_rel_perc_10  53784 non-null  float64
 6   ret_21           53685 non-null  float64
 7   ret_rel_perc_21  53685 non-null  float64
 8   ret_63           53307 non-null  float64
 9   ret_rel_perc_63  53307 non-null  float64
 10  PPO              53649 non-null  float64
 11  NATR             53748 non-null  float64
 12  RSI              53748 non-null  float64
 13  bbl              53838 non-null  float64
 14  bbu              53838 non-null  float64
 15  weekday          53874 non-n

In [33]:
data.to_hdf('data/sector_data.h5', 'features_data')